# 🧪 PyTorch Lab 9: Intro to Deep RL

This notebook is a **very simple introduction** to Reinforcement Learning (RL) concepts **without learning** a policy.

Goal: understand the interaction loop:

\[
(s_t) \xrightarrow{a_t} (r_t, s_{t+1}, \text{done})
\]

We will:
1. Build a tiny Maze environment (same as your later notebook).
2. Step through episodes with:
   - a **random policy**
   - a **hand-coded heuristic policy**
3. Inspect trajectories, returns, and what “episode termination” means.
4. Record a short rollout as a **GIF**.


## 0. Setup
We use `gymnasium` to wrap an environment, plus `matplotlib` for visualization and `imageio` for GIF export.


In [ ]:
!pip -q install gymnasium
!pip -q install imageio imageio-ffmpeg


In [ ]:
import os, glob
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import imageio
import matplotlib.pyplot as plt


## 1. Environment: a small maze with 4 actions

Actions:
- `0 = UP`
- `1 = DOWN`
- `2 = RIGHT`
- `3 = LEFT`

Observation:
- a grayscale 84×84 image (like Atari), so we can visualize easily.

Reward:
- `-0.01` every step (encourages shorter paths)
- `-0.1` if you bump into a wall
- `+1.0` if you reach the goal

Episode ends if:
- you reach the goal (`terminated=True`), or
- you hit the step limit (`truncated=True`)


In [ ]:
class SimpleMaze10x10_4Actions(gym.Env):
    metadata = {"render_modes": ["rgb_array"], "render_fps": 30}

    def __init__(self, size=10, max_steps=200, seed=0, render_mode=None):
        super().__init__()
        self.size = size
        self.max_steps = max_steps
        self.rng = np.random.default_rng(seed)
        self.render_mode = render_mode

        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=0, high=255, shape=(84, 84), dtype=np.uint8)

        self._build_fixed_maze()
        self.reset(seed=seed)

    def get_action_meanings(self):
        return ["UP", "DOWN", "RIGHT", "LEFT"]

    def _action_to_delta(self, a: int):
        if a == 0: return (-1, 0)  # UP
        if a == 1: return ( 1, 0)  # DOWN
        if a == 2: return ( 0, 1)  # RIGHT
        if a == 3: return ( 0,-1)  # LEFT
        raise ValueError(a)

    def _build_fixed_maze(self):
        s = self.size
        grid = np.zeros((s, s), dtype=np.uint8)

        # border walls
        grid[0, :] = 1
        grid[-1, :] = 1
        grid[:, 0] = 1
        grid[:, -1] = 1

        # internal walls (fixed layout)
        walls = [
            (2, 3), (2, 4),(2, 5),(2, 6),(2, 7),(2, 8),
            (3, 3),
            (5, 4), (5, 5), (5, 6),
            (6, 4),(6, 6),
            (7,4), (7, 5),(7, 6),
        ]
        for r, c in walls:
            grid[r, c] = 1

        self.grid = grid
        self.start = (1, 1)
        self.goal = (s - 2, s - 2)

        self.grid[self.start] = 0
        self.grid[self.goal] = 0

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.pos = list(self.start)
        self.steps = 0
        obs = self._get_obs()
        info = {"pos": tuple(self.pos)}
        return obs, info

    def step(self, action):
        self.steps += 1

        dr, dc = self._action_to_delta(int(action))
        nr, nc = self.pos[0] + dr, self.pos[1] + dc

        reward = -0.01
        bumped = False

        if self.grid[nr, nc] == 1:
            bumped = True
            reward -= 0.1
        else:
            self.pos = [nr, nc]

        terminated = (tuple(self.pos) == self.goal)
        if terminated:
            reward = 1.0

        truncated = (self.steps >= self.max_steps)
        obs = self._get_obs()
        info = {"pos": tuple(self.pos), "bumped": bumped}
        return obs, reward, terminated, truncated, info

    def _get_obs(self):
        H, W = 84, 84
        img = np.zeros((H, W), dtype=np.uint8)

        cell_h = H // self.size
        cell_w = W // self.size

        # walls
        for r in range(self.size):
            for c in range(self.size):
                if self.grid[r, c] == 1:
                    y0, y1 = r * cell_h, (r + 1) * cell_h
                    x0, x1 = c * cell_w, (c + 1) * cell_w
                    img[y0:y1, x0:x1] = 60

        # goal
        gr, gc = self.goal
        y0, y1 = gr * cell_h, (gr + 1) * cell_h
        x0, x1 = gc * cell_w, (gc + 1) * cell_w
        img[y0:y1, x0:x1] = 160

        # agent
        ar, ac = self.pos
        y0, y1 = ar * cell_h, (ar + 1) * cell_h
        x0, x1 = ac * cell_w, (ac + 1) * cell_w
        img[y0:y1, x0:x1] = 255

        return img

    def render(self):
        if self.render_mode == "rgb_array":
            g = self._get_obs()
            return np.stack([g, g, g], axis=-1)
        return None

    def close(self):
        pass


## 2. First contact: reset, inspect the observation, inspect actions


In [ ]:
env = SimpleMaze10x10_4Actions(seed=0)

obs, info = env.reset()
print("Start position:", info["pos"])
print("Action meanings:", env.get_action_meanings())
print("Observation shape:", obs.shape, obs.dtype)

plt.figure(figsize=(4,4))
plt.imshow(obs, cmap="gray", vmin=0, vmax=255)
plt.title("Initial observation (start)")
plt.axis("off")
plt.show()

env.close()


## 3. The RL interaction loop (one episode, random policy)

A **policy** is just a rule that chooses actions.
Here: pick a random action at each step.


In [ ]:
def run_episode_random(env, max_steps=50, seed=None):
    obs, info = env.reset(seed=seed)
    traj = []
    total_return = 0.0

    for t in range(max_steps):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        traj.append({
            "t": t,
            "action": action,
            "action_name": env.get_action_meanings()[action],
            "reward": reward,
            "pos": info["pos"],
            "bumped": info.get("bumped", False),
            "done": done,
            "terminated": terminated,
            "truncated": truncated,
        })
        total_return += reward
        if done:
            break

    return traj, total_return


In [ ]:
env = SimpleMaze10x10_4Actions(seed=0, max_steps=200)
traj, G = run_episode_random(env, max_steps=80, seed=0)

print("Episode length:", len(traj))
print("Return (sum of rewards):", round(G, 3))
print("Last transition:", traj[-1])

env.close()


### Visualize a few steps
We show a few frames to see movement.


In [ ]:
env = SimpleMaze10x10_4Actions(seed=0, max_steps=200)
obs, info = env.reset(seed=0)

frames = [obs]
for step in traj:
    obs, reward, terminated, truncated, info = env.step(step["action"])
    frames.append(obs)
    if step["done"]:
        break

show_steps = [0, 1, 2, 5, 10, 20]
plt.figure(figsize=(12,6))
for i, t in enumerate(show_steps):
    if t < len(frames):
        plt.subplot(2, 3, i+1)
        plt.imshow(frames[t], cmap="gray", vmin=0, vmax=255)
        plt.title(f"t={t}")
        plt.axis("off")
plt.suptitle("Random rollout snapshots")
plt.tight_layout()
plt.show()

env.close()


## 4. A hand-coded heuristic policy (not learning)

We write a simple “goal-seeking” policy:
- try to move RIGHT / DOWN toward the goal
- avoid walls by choosing the first valid move among candidates


In [ ]:
def greedy_towards_goal_candidates(env):
    r, c = env.pos
    gr, gc = env.goal

    candidates = []
    if gc > c: candidates.append(2)  # RIGHT
    if gr > r: candidates.append(1)  # DOWN
    if gc < c: candidates.append(3)  # LEFT
    if gr < r: candidates.append(0)  # UP

    for a in [2, 1, 3, 0]:  # fallback order
        if a not in candidates:
            candidates.append(a)

    return candidates


def run_episode_heuristic(env, max_steps=200, seed=None):
    obs, info = env.reset(seed=seed)
    traj = []
    total_return = 0.0

    for t in range(max_steps):
        chosen = None
        for a in greedy_towards_goal_candidates(env):
            dr, dc = env._action_to_delta(a)
            nr, nc = env.pos[0] + dr, env.pos[1] + dc
            if env.grid[nr, nc] == 0:
                chosen = a
                break
        if chosen is None:
            chosen = env.action_space.sample()

        obs, reward, terminated, truncated, info = env.step(chosen)
        done = terminated or truncated

        traj.append({
            "t": t,
            "action": chosen,
            "action_name": env.get_action_meanings()[chosen],
            "reward": reward,
            "pos": info["pos"],
            "bumped": info.get("bumped", False),
            "done": done,
        })
        total_return += reward
        if done:
            break

    return traj, total_return


In [ ]:
env = SimpleMaze10x10_4Actions(seed=0, max_steps=200)
traj_h, G_h = run_episode_heuristic(env, seed=0)

print("Heuristic episode length:", len(traj_h))
print("Heuristic return:", round(G_h, 3))
print("Reached goal?", traj_h[-1]["pos"] == env.goal)

env.close()


## 5. Compare policies statistically (no learning)

We run many episodes and compare:
- mean return
- mean episode length
- success rate (reach goal)


In [ ]:
def evaluate(policy_runner, n_episodes=50, seed0=0):
    returns, lengths = [], []
    successes = 0

    env = SimpleMaze10x10_4Actions(seed=seed0, max_steps=200)
    for i in range(n_episodes):
        traj, G = policy_runner(env, seed=seed0+i)
        returns.append(G)
        lengths.append(len(traj))
        if traj and traj[-1]["pos"] == env.goal:
            successes += 1
    env.close()

    return {
        "mean_return": float(np.mean(returns)),
        "std_return": float(np.std(returns)),
        "mean_length": float(np.mean(lengths)),
        "success_rate": successes / n_episodes,
    }

stats_random = evaluate(lambda env, seed: run_episode_random(env, max_steps=200, seed=seed), n_episodes=50, seed0=0)
stats_heur  = evaluate(lambda env, seed: run_episode_heuristic(env, max_steps=200, seed=seed), n_episodes=50, seed0=0)

stats_random, stats_heur


In [ ]:
labels = ["Random", "Heuristic"]
mean_returns = [stats_random["mean_return"], stats_heur["mean_return"]]
mean_lengths = [stats_random["mean_length"], stats_heur["mean_length"]]
success = [stats_random["success_rate"], stats_heur["success_rate"]]

plt.figure()
plt.bar(labels, mean_returns)
plt.title("Mean return (higher is better)")
plt.ylabel("Return")
plt.show()

plt.figure()
plt.bar(labels, mean_lengths)
plt.title("Mean episode length (lower is better)")
plt.ylabel("Steps")
plt.show()

plt.figure()
plt.bar(labels, success)
plt.title("Success rate (reach goal)")
plt.ylabel("Rate")
plt.ylim(0, 1)
plt.show()


## 6. Record one rollout as a GIF

We record an episode using `RecordVideo` (mp4) then convert to gif.


In [ ]:
def newest_mp4(folder):
    mp4s = sorted(glob.glob(os.path.join(folder, "*.mp4")), key=os.path.getmtime)
    if not mp4s:
        raise FileNotFoundError(f"No mp4 found in {folder}")
    return mp4s[-1]

def mp4_to_gif(mp4_path, gif_path, fps=30):
    reader = imageio.get_reader(mp4_path)
    frames = [frame for frame in reader]
    reader.close()
    imageio.mimsave(gif_path, frames, fps=fps)


In [ ]:
VIDEO_DIR = "videos_intro"
os.makedirs(VIDEO_DIR, exist_ok=True)

env = SimpleMaze10x10_4Actions(seed=0, max_steps=200, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder=VIDEO_DIR, episode_trigger=lambda i: True)

traj_gif, G_gif = run_episode_heuristic(env, seed=0)
env.close()

mp4_path = newest_mp4(VIDEO_DIR)
gif_path = "intro_heuristic.gif"
mp4_to_gif(mp4_path, gif_path, fps=30)

print("Heuristic return:", round(G_gif, 3))
print("Saved gif:", gif_path)


## Exercises (optional)

1. Modify the maze: add/remove a wall and see if the heuristic still works.
2. Change the reward shaping: increase wall penalty and see how random returns change.
3. Design a better hand-coded policy (wall-following, memory of visited cells).
4. Add randomness: with probability 0.1, take a random action even in the heuristic policy.
